In [1]:
%cd hidden

/raid/home/ashhar21137/watermarking3/stable_signature/hidden


In [2]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]='2'

In [3]:
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# you should run this notebook in the root directory of the hidden project for the following imports to work
# %cd ..
from models import HiddenEncoder, HiddenDecoder, EncoderWithJND, EncoderDecoder
from attenuations import JND
from PIL import Image 
import numpy as np 
from tqdm import tqdm 
from skimage.util import random_noise
from torchvision import transforms
from collections import defaultdict
import re
import cv2
from PIL import Image, ImageEnhance
from diffusers import AutoPipelineForText2Image
from diffusers import AutoPipelineForImage2Image
import json

In [4]:
def msg2str(msg):
    return "".join([('1' if el else '0') for el in msg])

def str2msg(str):
    return [True if el=='1' else False for el in str]

In [5]:
class Params():
    def __init__(self, encoder_depth:int, encoder_channels:int, decoder_depth:int, decoder_channels:int, num_bits:int,
                attenuation:str, scale_channels:bool, scaling_i:float, scaling_w:float):
        # encoder and decoder parameters
        self.encoder_depth = encoder_depth
        self.encoder_channels = encoder_channels
        self.decoder_depth = decoder_depth
        self.decoder_channels = decoder_channels
        self.num_bits = num_bits
        # attenuation parameters
        self.attenuation = attenuation
        self.scale_channels = scale_channels
        self.scaling_i = scaling_i
        self.scaling_w = scaling_w

NORMALIZE_IMAGENET = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
UNNORMALIZE_IMAGENET = transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225], std=[1/0.229, 1/0.224, 1/0.225])
default_transform = transforms.Compose([transforms.ToTensor(), NORMALIZE_IMAGENET])

params = Params(
    encoder_depth=4, encoder_channels=64, decoder_depth=8, decoder_channels=64, num_bits=48,
    attenuation="jnd", scale_channels=False, scaling_i=1, scaling_w=1.5
)

decoder = HiddenDecoder(
    num_blocks=params.decoder_depth,
    num_bits=params.num_bits,
    channels=params.decoder_channels
)
encoder = HiddenEncoder(
    num_blocks=params.encoder_depth,
    num_bits=params.num_bits,
    channels=params.encoder_channels
)
attenuation = JND(preprocess=UNNORMALIZE_IMAGENET) if params.attenuation == "jnd" else None
encoder_with_jnd = EncoderWithJND(
    encoder, attenuation, params.scale_channels, params.scaling_i, params.scaling_w
)

In [6]:
ckpt_path = "ckpts/hidden_replicate.pth"

state_dict = torch.load(ckpt_path, map_location='cpu')['encoder_decoder']
encoder_decoder_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
encoder_state_dict = {k.replace('encoder.', ''): v for k, v in encoder_decoder_state_dict.items() if 'encoder' in k}
decoder_state_dict = {k.replace('decoder.', ''): v for k, v in encoder_decoder_state_dict.items() if 'decoder' in k}

encoder.load_state_dict(encoder_state_dict)
decoder.load_state_dict(decoder_state_dict)

encoder_with_jnd = encoder_with_jnd.to(device).eval()
decoder = decoder.to(device).eval()

In [7]:
paraphrase_model_id = 'stabilityai/stable-diffusion-xl-base-1.0'
# model_id = 'stabilityai/stable-diffusion-2-1-base'

pipeline_text2image = AutoPipelineForText2Image.from_pretrained(paraphrase_model_id, torch_dtype=torch.float16, variant="fp16", use_safetensors=True, add_watermarker=False).to(device)
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to(device)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
img_dir = "/raid/home/ashhar21137/watermarking3/stable_sig_watermarked"

In [9]:
with open('/raid/home/ashhar21137/watermarking3/stable_signature/captions_train2014_new_format.json','r') as file : 
    new = json.load(file)

In [10]:
img_ids = []
img_pths = []
imgs = os.listdir(img_dir)
for i in imgs : 
    if '.png' or '.jpg' in i :
        id = re.split('[_.]',i)[1]
        img_ids.append(id) 
        img_pths.append(os.path.join(img_dir,i))



In [11]:
print("Imgs ids : ", img_ids)


Imgs ids :  ['79481', '96557', '280260', '62279', '170558', '215481', '520208', '95989', '504616', '309322', '239728', '341550', '405613', '441488', '420721', '18885', '399956', '93785', '570246', '262845', '49183', '142088', '394326', '524068', '352884', '199628', '160559', '57870', '398494', '241364', '84427', '393493', '483008', '469605', '348850', '387083', '77375', '122108', '540162', '19380', '44404', '345781', '405207', '153946', '457732', '153674', '472925', '388754', '170850', '384029', '62604', '570608', '317237', '299411', '109424', '581674', '239811', '318189', '278347', '90570', '374114', '505126', '8649', '145179', '66514', '462559', '4428', '210932', '257350', '122688', '377694', '438422', '161919', '354444', '287541', '556801', '186233', '95133', '496939', '71631', '371029', '259446', '55627', '334080', '150867', '470072', '181104', '559527', '209967', '444546', '409513', '165499', '351430', '36633', '147303', '474882', '318574', '222016', '236772', '486320', '551125', 

In [12]:
print("Images : ",imgs)


Images :  ['Image_79481.jpg', 'Image_96557.jpg', 'Image_280260.jpg', 'Image_62279.jpg', 'Image_170558.jpg', 'Image_215481.jpg', 'Image_520208.jpg', 'Image_95989.jpg', 'Image_504616.jpg', 'Image_309322.jpg', 'Image_239728.jpg', 'Image_341550.jpg', 'Image_405613.jpg', 'Image_441488.jpg', 'Image_420721.jpg', 'Image_18885.jpg', 'Image_399956.jpg', 'Image_93785.jpg', 'Image_570246.jpg', 'Image_262845.jpg', 'Image_49183.jpg', 'Image_142088.jpg', 'Image_394326.jpg', 'Image_524068.jpg', 'Image_352884.jpg', 'Image_199628.jpg', 'Image_160559.jpg', 'Image_57870.jpg', 'Image_398494.jpg', 'Image_241364.jpg', 'Image_84427.jpg', 'Image_393493.jpg', 'Image_483008.jpg', 'Image_469605.jpg', 'Image_348850.jpg', 'Image_387083.jpg', 'Image_77375.jpg', 'Image_122108.jpg', 'Image_540162.jpg', 'Image_19380.jpg', 'Image_44404.jpg', 'Image_345781.jpg', 'Image_405207.jpg', 'Image_153946.jpg', 'Image_457732.jpg', 'Image_153674.jpg', 'Image_472925.jpg', 'Image_388754.jpg', 'Image_170850.jpg', 'Image_384029.jpg', '

In [19]:
count = 0 
for i in range(len(img_ids)) : 
    id = re.split('[_.]',imgs[i])[1]
    if(img_ids[i] != id ) : 
        print(img_ids)
        count += 1

print(count)

0


In [20]:
wmis = os.listdir(img_dir) 
print("Wmis : ",wmis)


Wmis :  ['Image_79481.jpg', 'Image_96557.jpg', 'Image_280260.jpg', 'Image_62279.jpg', 'Image_170558.jpg', 'Image_215481.jpg', 'Image_520208.jpg', 'Image_95989.jpg', 'Image_504616.jpg', 'Image_309322.jpg', 'Image_239728.jpg', 'Image_341550.jpg', 'Image_405613.jpg', 'Image_441488.jpg', 'Image_420721.jpg', 'Image_18885.jpg', 'Image_399956.jpg', 'Image_93785.jpg', 'Image_570246.jpg', 'Image_262845.jpg', 'Image_49183.jpg', 'Image_142088.jpg', 'Image_394326.jpg', 'Image_524068.jpg', 'Image_352884.jpg', 'Image_199628.jpg', 'Image_160559.jpg', 'Image_57870.jpg', 'Image_398494.jpg', 'Image_241364.jpg', 'Image_84427.jpg', 'Image_393493.jpg', 'Image_483008.jpg', 'Image_469605.jpg', 'Image_348850.jpg', 'Image_387083.jpg', 'Image_77375.jpg', 'Image_122108.jpg', 'Image_540162.jpg', 'Image_19380.jpg', 'Image_44404.jpg', 'Image_345781.jpg', 'Image_405207.jpg', 'Image_153946.jpg', 'Image_457732.jpg', 'Image_153674.jpg', 'Image_472925.jpg', 'Image_388754.jpg', 'Image_170850.jpg', 'Image_384029.jpg', 'Im

In [21]:
save_dir = "/raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15"

In [22]:
# paraphrase_detection = defaultdict(lambda: defaultdict(dict))


In [23]:
# print(len(wmis))
# print(len(img_ids))
# print(len(imgs))

In [28]:
paraphrase_detection = defaultdict(lambda: defaultdict(dict))

In [29]:
def load_image(image_path):
    """
    Load an image from the specified file path using PIL.

    Args:
    image_path (str): Path to the image file.

    Returns:
    PIL.Image.Image: Loaded image object.
    """
    return Image.open(image_path)

In [30]:
count = 1
for i in tqdm(range(len(wmis))) :

    # if count > 5 : break 

    print(f"---------- Count = {count} : Visual Paraphrasing for the watermarked version of {img_ids[i]}-------------")
    print()

    # print(f"records_512_st_0.35/{img_ids[0]}.txt")
    print("Image name : ",wmis[i])
    
    id = re.split('[_.]',wmis[i])[1]

    print("Image id :",id)
    # print("Image id : ",img_ids[i])

    paraphrase_detection[id]["name"] = wmis[i]

    captions = new['annotations'][id]


    post_img = os.path.join(img_dir,wmis[i]) # watermarked image path 
    image = Image.open(post_img)
    # init_image = load_image(image)
    init_image = image

    gen_image = pipeline(captions, image=init_image, strength=0.1, guidance_scale=7.5).images
    
    # image
    # make_image_grid([init_image, image], rows=2, cols=2)

    directory =  f'{save_dir}/{id}'
    print(f"Saving generated images at {directory}")
    if not os.path.exists(directory):
        os.makedirs(directory)

    for k in range(len(gen_image)) :
        gen_save_dir = os.path.join(directory,f'{id}_gen_{k}.png')
        gen_image[k].save(gen_save_dir)
        print(f"Generated Image saved to {gen_save_dir}")

    # is_watermarked
    num_images = len(gen_image) # 5 images generated from 5 captions for each input image
    print()
    print("Number of images generated : ",num_images)
    # print("********** Watermark detection for generated images without Captions ***************")     

    # avg_no_prompt = 0 
    # num_detected = 0 
    # for j in range(num_images):
    #     post_img = os.path.join(directory,f'{id}_gen_{j}.png')
    #     tester_prompt = '' # assume at the detection time, the original prompt is unknown
    #     text_embeddings = pipe.get_text_embedding(tester_prompt)
    #     det_prob = 1 - watermark_prob(post_img, pipe, wm_pipe, text_embeddings)
        
    #     avg_no_prompt = avg_no_prompt + det_prob
    #     num_detected = num_detected = (det_prob>0.9)
        
    #     # logging.info(f'Watermark Presence Prob.: {det_prob}')
    #     print(f'Watermark Presence Prob.: {det_prob}')

    # paraphrase_detection[id]["without_captions_avg_det_prob"] = avg_no_prompt/num_images
    # paraphrase_detection[id]["without_captions_det_rate"] = num_detected/num_images


    avg_with_prompt = 0 
    num_detected = 0


    print()
    print("********** Watermark detection for generated images ***************")  

    for j in range(num_images):
        post_img = os.path.join(directory, f'{id}_gen_{j}.png')
        # tester_prompt = '' # assume at the detection time, the original prompt is unknown
        print(f"caption : {captions[j]}")

        # Load the image
        img = Image.open(post_img).convert('RGB')
        img = img.resize((512, 512), Image.BICUBIC)
        img_pt = default_transform(img).unsqueeze(0).to(device)
        
        # Decode the watermark
        ft = decoder(img_pt)
        decoded_msg = ft > 0  # b k -> b k
        
        # Assume you have the original message or compute its bit accuracy
        msg_ori = torch.Tensor(str2msg("111010110101000001010111010011010100010000100111")).unsqueeze(0).to(device)  # Example message
        accs = (~torch.logical_xor(decoded_msg, msg_ori))  # b k -> b k
        bit_accuracy = accs.sum().item() / params.num_bits

        # print(f"Message: {msg2str(msg_ori.squeeze(0).cpu().numpy())}")
        # print(f"Decoded: {msg2str(decoded_msg.squeeze(0).cpu().numpy())}")
        # print(f"Bit Accuracy: {bit_accuracy}")
        
        detection_result = 1 if bit_accuracy > 0.90 else 0
        avg_with_prompt = avg_with_prompt + bit_accuracy
        num_detected = num_detected + detection_result

        print(f'Watermark Presence Prob.: {bit_accuracy}')    
        paraphrase_detection[id]["avg_det_prob"] = avg_with_prompt/num_images
        paraphrase_detection[id]["avg_det_rate"] = num_detected/num_images

    # break
    count = count + 1  


  0%|          | 0/200 [00:00<?, ?it/s]

---------- Count = 1 : Visual Paraphrasing for the watermarked version of 79481-------------

Image name :  Image_79481.jpg
Image id : 79481


  0%|          | 0/5 [00:00<?, ?it/s]

Saving generated images at /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/79481
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/79481/79481_gen_0.png
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/79481/79481_gen_1.png
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/79481/79481_gen_2.png
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/79481/79481_gen_3.png
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/79481/79481_gen_4.png

Number of images generated :  5

********** Watermark detection for generated images ***************
caption : A blue motor cycle parked in front of a house next to trash cans.
Watermark Presence Prob.: 0.4791666666666667
caption : A blue motorcycle under an umbrella and being held up with a 

  0%|          | 1/200 [00:02<07:08,  2.15s/it]

...

Watermark Presence Prob.: 0.4791666666666667
caption : police officers in uniforms standing on a strait line
Watermark Presence Prob.: 0.4791666666666667
caption : Several police officer lined up in front of motorcycles.
Watermark Presence Prob.: 0.4791666666666667
caption : a line of police men  that are posing for a picture
Watermark Presence Prob.: 0.4583333333333333
---------- Count = 200 : Visual Paraphrasing for the watermarked version of 454325-------------

Image name :  Image_454325.jpg
Image id : 454325


  0%|          | 0/5 [00:00<?, ?it/s]

Saving generated images at /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/454325
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/454325/454325_gen_0.png
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/454325/454325_gen_1.png
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/454325/454325_gen_2.png
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/454325/454325_gen_3.png
Generated Image saved to /raid/home/ashhar21137/watermarking3/stable_signature/ss_paraphrase_0.15/454325/454325_gen_4.png

Number of images generated :  5

********** Watermark detection for generated images ***************
caption : A modern restroom is equipped with fashionable sinks and urinals surrounded by architectural subway tile.
Watermark Presence Prob.: 0.6041666666666666
caption : a bathroom

100%|██████████| 200/200 [07:15<00:00,  2.18s/it]

Watermark Presence Prob.: 0.625


In [31]:
paraphrase_detection

defaultdict(<function __main__.<lambda>()>,
            {'79481': defaultdict(dict,
                         {'name': 'Image_79481.jpg',
                          'avg_det_prob': 0.5208333333333333,
                          'avg_det_rate': 0.0}),
             '96557': defaultdict(dict,
                         {'name': 'Image_96557.jpg',
                          'avg_det_prob': 0.5041666666666667,
                          'avg_det_rate': 0.0}),
             '280260': defaultdict(dict,
                         {'name': 'Image_280260.jpg',
                          'avg_det_prob': 0.5166666666666667,
                          'avg_det_rate': 0.0}),
             '62279': defaultdict(dict,
                         {'name': 'Image_62279.jpg',
                          'avg_det_prob': 0.5958333333333333,
                          'avg_det_rate': 0.0}),
             '170558': defaultdict(dict,
                         {'name': 'Image_170558.jpg',
                          'avg_det_prob': 0

In [32]:
len(paraphrase_detection)

200

In [33]:
with open("/raid/home/ashhar21137/watermarking3/stable_signature/stable_sig_0.15_paraphrased.json","w") as file : 
    json.dump(paraphrase_detection,file,indent=4)
